In [ ]:
from dotenv import load_dotenv
import os
load_dotenv(dotenv_path=".env", override=True)

In [ ]:
## Tools

from langchain_openai import ChatOpenAI
from pydantic import BaseModel,Field
import openai
from langsmith.wrappers import wrap_openai
from langsmith import traceable
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
import requests

llm = ChatOpenAI(model="gpt-4o")

# Create specialized agents

@tool
def fetch_emp_data(username:str, password :str) -> dict:
    """
    Fetches employee data from an external API.

    Parameters:
        username (str) : An username of an employee. This can be an email id as well
        password (str) : A passphrase used to authenticate an employee

    Returns:
        dict: A dictionary with employee details.
    """
    payload = {
        "username": username.lower(),
        "password": password
    }
    base_url = "http://127.0.0.1:5000/find_details"
    
    print(f"--- Calling external API to fetch leave data for username: {username} ---")
    
    try:
        response = requests.post(base_url,data=payload)
        data = response.json()
        print (data)
        return data
    except requests.exceptions.ConnectionError as e:
        print(f"\nError: Could not connect to the Flask server.")
        print(f"Error details: {e}")
    except requests.exceptions.HTTPError as e:
        print(f"\nError: HTTP request failed with status code {response.status_code}")
        print(f"Error details: {e}")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")



In [ ]:
## Agents

employee_agent = create_react_agent(
    model=llm,
    tools=[fetch_emp_data],
    name="employee_agent",
    prompt="You are an agent that pulls employee information from an external API"
)




In [ ]:
#Compile and run

result = employee_agent.invoke({
    "messages": [
        {
            "role": "user",
            "content": "Can you find the number of remaining sick leaves for the employee with username = risaha@redhat.com and password = Hello!123 ? "
        }
    ]
})

print(result)

In [ ]:
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage

for msg in result['messages']:    
    msg.pretty_print()

In [ ]:
for msg in result['messages']:    
  
    if isinstance(msg, ToolMessage) :
        print(f"Tool Message name: {msg.name}")
        print(f"Tool Message content: {msg.content}")